In [1]:
import pandas as pd
import numpy as np
import holidays
import calendar
from datetime import date
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [ ]:
# Load the original hourly data
DATA_PATH = "merged.csv"
TIME_COLUMN = 'measured_at'
GROUP_COLUMN = 'group_id'
TARGET_COLUMN = 'consumption'

print(f"Loading full hourly dataset from {DATA_PATH}...")
df = pd.read_csv(DATA_PATH)
df[TIME_COLUMN] = pd.to_datetime(df[TIME_COLUMN])

# Define our aggregations
weather_cols = ['avg_temp', 'avg_hum', 'wind', 'rain', 'air_pressure']
price_cols = ['eur_per_mwh']
static_cols = ['m_area', 'region', 'municipality', 'segment', 'p_type', 'c_bucket']

# Create a dictionary of aggregations
aggs = {}
aggs[TARGET_COLUMN] = 'sum'
for col in price_cols + weather_cols:
    aggs[col] = 'mean'
for col in static_cols:
    aggs[col] = 'first'

# Set index and resample
print("Aggregating hourly data to monthly...")
df = df.set_index(TIME_COLUMN)

# Group by group_id, then resample by Month Start ('MS')
monthly_df = df.groupby(GROUP_COLUMN).resample('MS').agg(aggs)

# Clean up the new DataFrame
monthly_df = monthly_df.reset_index()
monthly_df = monthly_df.dropna(subset=[TARGET_COLUMN])

print(f"Aggregation complete. New monthly dataset has {len(monthly_df)} rows.")
monthly_df.info()

Loading full hourly dataset from merged.csv...
Aggregating hourly data to monthly...
Aggregation complete. New monthly dataset has 5040 rows.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5040 entries, 0 to 5039
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   group_id      5040 non-null   int64              
 1   measured_at   5040 non-null   datetime64[ns, UTC]
 2   consumption   5040 non-null   float64            
 3   eur_per_mwh   5040 non-null   float64            
 4   avg_temp      5040 non-null   float64            
 5   avg_hum       5040 non-null   float64            
 6   wind          5040 non-null   float64            
 7   rain          5040 non-null   float64            
 8   air_pressure  5040 non-null   float64            
 9   m_area        5040 non-null   object             
 10  region        5040 non-null   object             
 11  municipality  5040 non-null   o

In [ ]:
print("--- Starting Monthly Feature Engineering (Direct Strategy) ---")
data_full = monthly_df.copy()

# Sort for correct lag creation
data_full = data_full.sort_values(by=[GROUP_COLUMN, TIME_COLUMN])

# Create PREDICTOR Features (Our 'X' variables)
print("Creating time features...")
data_full['month'] = data_full[TIME_COLUMN].dt.month
data_full['year'] = data_full[TIME_COLUMN].dt.year
data_full['month_sin'] = np.sin(2 * np.pi * data_full['month'] / 12.0)
data_full['month_cos'] = np.cos(2 * np.pi * data_full['month'] / 12.0)

fin_holidays = holidays.Finland()
def count_holidays_in_month(year, month):
    if not (isinstance(year, int) and isinstance(month, int)): return 0
    try:
        _, num_days = calendar.monthrange(year, month)
        count = 0
        for day in range(1, num_days + 1):
            if date(year, month, day) in fin_holidays:
                count += 1
        return count
    except ValueError:
        return 0
vectorized_holiday_count = np.vectorize(count_holidays_in_month)
data_full['num_holidays'] = vectorized_holiday_count(data_full['year'], data_full['month'])
data_full = data_full.drop(columns=['month'])

print("Creating lag and rolling features...")
df_grouped = data_full.groupby(GROUP_COLUMN)
lags = [1, 2, 3, 12]
windows = [12]
for lag in lags:
    data_full[f'consumption_lag_{lag}m'] = df_grouped['consumption'].shift(lag)
for window in windows:
    rolling_feat = df_grouped['consumption'].rolling(window=window, min_periods=1).mean()
    data_full[f'consumption_roll_mean_{window}m'] = rolling_feat.reset_index(level=0, drop=True)

exog_cols = ['eur_per_mwh', 'avg_temp', 'avg_hum', 'wind', 'rain', 'air_pressure']
for col in exog_cols:
    if col in data_full.columns:
        data_full[f'{col}_lag_12m'] = df_grouped[col].shift(12)

# Create TARGET Features (Our 'y' variables)
print("Creating 12 future-month target columns...")
HORIZON = 12
target_cols = []
for i in range(1, HORIZON + 1):
    target_name = f'target_{i}m_ahead'
    data_full[target_name] = df_grouped['consumption'].shift(-i)
    target_cols.append(target_name)

# Final Cleanup
for col in static_cols:
    data_full[col] = data_full[col].astype('category')

# Define our final features list (used by all cells now)
static_features = [col for col in data_full.columns if col in static_cols]
dynamic_features = [
    'year', 'month_sin', 'month_cos', 'num_holidays',
    'consumption_lag_1m', 'consumption_lag_2m', 'consumption_lag_3m', 'consumption_lag_12m',
    'consumption_roll_mean_12m',
    'eur_per_mwh_lag_12m', 'avg_temp_lag_12m', 'avg_hum_lag_12m', 
    'wind_lag_12m', 'rain_lag_12m', 'air_pressure_lag_12m'
]
features = dynamic_features + static_features

print(f"--- Monthly Feature Engineering Complete ---")
data_full.info()

--- Starting Monthly Feature Engineering (Direct Strategy) ---
Creating time features...
Creating lag and rolling features...
Creating 12 future-month target columns...
--- Monthly Feature Engineering Complete ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5040 entries, 0 to 5039
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   group_id                   5040 non-null   int64              
 1   measured_at                5040 non-null   datetime64[ns, UTC]
 2   consumption                5040 non-null   float64            
 3   eur_per_mwh                5040 non-null   float64            
 4   avg_temp                   5040 non-null   float64            
 5   avg_hum                    5040 non-null   float64            
 6   wind                       5040 non-null   float64            
 7   rain                       5040 non-null   float64            

In [ ]:
static_features = [col for col in data_full.columns if col in static_cols]

dynamic_features = [
    'year', 'month_sin', 'month_cos', 'num_holidays',
    
    'consumption_lag_12m',
    'eur_per_mwh_lag_12m',
    'avg_temp_lag_12m', 'avg_hum_lag_12m', 'wind_lag_12m',
    'rain_lag_12m', 'air_pressure_lag_12m'
]

features = dynamic_features + static_features
print(f"--- Training with a non-leaking {len(features)} feature set ---")

# Create Train/Validation Split
train_val_data = data_full.dropna(subset=features + target_cols)

# Ensure the validation period is 12 months at the end
val_cutoff = train_val_data[TIME_COLUMN].max() - pd.DateOffset(months=12)
train = train_val_data[train_val_data[TIME_COLUMN] <= val_cutoff].copy()
val = train_val_data[train_val_data[TIME_COLUMN] > val_cutoff].copy()

print(f"Training on {train[TIME_COLUMN].min()} to {train[TIME_COLUMN].max()}")
print(f"Validating on {val[TIME_COLUMN].min()} to {val[TIME_COLUMN].max()}")

X_train = train[features]
X_val = val[features]

# Define the Optuna Objective Function
TARGET_TO_TUNE = 'target_6m_ahead'
y_train_tune = train[TARGET_TO_TUNE]
y_val_tune = val[TARGET_TO_TUNE]

tune_train_data = lgb.Dataset(X_train, label=y_train_tune, categorical_feature=static_features)
tune_val_data = lgb.Dataset(X_val, label=y_val_tune, reference=tune_train_data)

def objective(trial):
    params = {
        'objective': 'regression_l1',
        'metric': 'rmse',
        'seed': 42,
        'n_jobs': -1,
        'verbose': -1,
        'n_estimators': 2000,
        'feature_pre_filter': False,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 10, 40),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.1, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.1, 10.0, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 30, 150),
    }
    model_tune = lgb.train(
        params,
        tune_train_data,
        valid_sets=[tune_val_data],
        callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
    )
    return model_tune.best_score['valid_0']['rmse']

# Run the Optuna Study
print(f"--- Starting Optuna Search (Tuning for {TARGET_TO_TUNE}) ---")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=150)

print("--- Search Complete ---")
print(f"Best Validation RMSE (for {TARGET_TO_TUNE}): {study.best_value:.2f}")
print("Best Parameters Found:")
print(study.best_params)

--- Training with a non-leaking 17 feature set ---
Training on 2022-01-01 00:00:00+00:00 to 2022-09-01 00:00:00+00:00
Validating on 2022-10-01 00:00:00+00:00 to 2023-09-01 00:00:00+00:00


In [ ]:
# best_params = {'learning_rate': 0.08056583009280904, 'num_leaves': 21, 'subsample': 0.9525650463826345, 'colsample_bytree': 0.850318972819252, 'lambda_l1': 6.918875094886156, 'lambda_l2': 0.3219122718364551, 'min_child_samples': 37}

# Get the best parameters from the study
best_params = study.best_params

# Set up final parameters
final_params = {
    'objective': 'regression_l1',
    'metric': 'rmse',
    'seed': 42,
    'n_jobs': -1,
    'verbose': -1,
    'n_estimators': 2000,
    'feature_pre_filter': False,
    **best_params  # Unpacks all the best params (lr, num_leaves, etc.)
}

# Train all 12 models in a loop
print("--- Training All 12 Models ---")
model_pipeline = {} # This will hold our 12 models
HORIZON = 12

for i in range(1, HORIZON + 1):
    target_name = f'target_{i}m_ahead'
    print(f"Training model for: {target_name}")
    
    y_train_model = train[target_name]
    y_val_model = val[target_name]
    
    train_data_model = lgb.Dataset(X_train, label=y_train_model, categorical_feature=static_features)
    val_data_model = lgb.Dataset(X_val, label=y_val_model, reference=train_data_model)
    
    model = lgb.train(
        final_params,
        train_data_model,
        valid_sets=[val_data_model],
        valid_names=['validation'],
        callbacks=[lgb.early_stopping(50, verbose=False)]
    )
    
    # Store the trained model
    model_pipeline[i] = model

print("\n--- Training Complete: 12 models stored in 'model_pipeline' ---")

--- Training All 12 Models ---
Training model for: target_1m_ahead
Training model for: target_2m_ahead
Training model for: target_3m_ahead
Training model for: target_4m_ahead
Training model for: target_5m_ahead
Training model for: target_6m_ahead
Training model for: target_7m_ahead
Training model for: target_8m_ahead
Training model for: target_9m_ahead
Training model for: target_10m_ahead
Training model for: target_11m_ahead
Training model for: target_12m_ahead

--- Training Complete: 12 models stored in 'model_pipeline' ---


In [ ]:
print("--- Starting Direct 12-Month Forecast (Corrected) ---")

# Get the last known row of data for EACH group
latest_history_df = data_full.sort_values(by=TIME_COLUMN).groupby(GROUP_COLUMN).tail(1)

# Extract the features that DON'T change (Lags, Statics)
lag_features = [
    'consumption_lag_1m', 'consumption_lag_2m', 'consumption_lag_3m', 'consumption_lag_12m',
    'consumption_roll_mean_12m',
    'eur_per_mwh_lag_12m', 'avg_temp_lag_12m', 'avg_hum_lag_12m', 
    'wind_lag_12m', 'rain_lag_12m', 'air_pressure_lag_12m'
]
base_features_df = latest_history_df[static_features + lag_features + [GROUP_COLUMN]]

print(f"Using historical lag features from {latest_history_df[TIME_COLUMN].max().strftime('%Y-%m')} for {len(base_features_df)} groups.")

# Create a list to hold our monthly forecast DataFrames
forecast_dfs = []
HORIZON = 12
last_train_date = latest_history_df[TIME_COLUMN].max()

# Loop 12 times and predict
for i in range(1, HORIZON + 1):
    model = model_pipeline[i]
    
    # The correct features for this future month
    current_date = last_train_date + pd.DateOffset(months=i)
    
    # A new DataFrame for this month's features
    X_future = base_features_df.copy()
    
    # The *correct* future time features for this month
    X_future[TIME_COLUMN] = current_date
    X_future['month'] = X_future[TIME_COLUMN].dt.month
    X_future['year'] = X_future[TIME_COLUMN].dt.year
    X_future['month_sin'] = np.sin(2 * np.pi * X_future['month'] / 12.0)
    X_future['month_cos'] = np.cos(2 * np.pi * X_future['month'] / 12.0)
    X_future['num_holidays'] = vectorized_holiday_count(X_future['year'], X_future['month'])
    
    X_future_final = X_future[features] 
    
    predictions = model.predict(X_future_final)
    
    month_forecast_df = pd.DataFrame({
        TIME_COLUMN: current_date,
        GROUP_COLUMN: base_features_df[GROUP_COLUMN].values,
        'consumption_pred': predictions
    })
    forecast_dfs.append(month_forecast_df)

final_forecast_df = pd.concat(forecast_dfs)

print("--- Direct Forecast Complete ---")
final_forecast_df.head()

--- Starting Direct 12-Month Forecast (Corrected) ---
Using historical lag features from 2024-09 for 112 groups.
--- Direct Forecast Complete ---


,measured_at,group_id,consumption_pred
0,2024-10-01 00:00:00+00:00,738,389.766449
1,2024-10-01 00:00:00+00:00,740,109.972470
2,2024-10-01 00:00:00+00:00,393,244.508926
3,2024-10-01 00:00:00+00:00,708,333.439418
4,2024-10-01 00:00:00+00:00,225,197.170310


In [ ]:
# Load the Test Set with True Values
TEST_TRUTH_PATH = "monthly_df_test.csv"
print(f"Loading true values from {TEST_TRUTH_PATH}...")
test_true_df = pd.read_csv(TEST_TRUTH_PATH)

# Convert 'measured_at' to datetime and *localize to UTC*
test_true_df[TIME_COLUMN] = pd.to_datetime(test_true_df[TIME_COLUMN], utc=True)

# Rename columns for a clean merge
forecast_df_renamed = final_forecast_df.rename(columns={'consumption': 'consumption_pred'})
test_true_df_renamed = test_true_df.rename(columns={'consumption': 'consumption_true'})

# Merge Forecasts and True Values 
keys = [TIME_COLUMN, GROUP_COLUMN]
score_df = pd.merge(
    forecast_df_renamed[keys + ['consumption_pred']],
    test_true_df_renamed[keys + ['consumption_true']],
    on=keys,
    how='inner'
)

# Calculate and Print Final Scores
if len(score_df) == 0:
    print("\n--- FATAL ERROR: Merge failed. No matching rows found. ---")
    print("Check that 'measured_at' (including UTC) and 'group_id' values match.")
elif len(score_df) < len(final_forecast_df):
    print(f"\nWarning: Scored {len(score_df)} rows, but forecast had {len(final_forecast_df)} rows.")
    print("Some 'group_id's or 'measured_at' dates may not have been in the test file.")
else:
    print(f"\nSuccessfully merged {len(score_df)} rows for scoring.")

# Calculate metrics
mae = mean_absolute_error(score_df['consumption_true'], score_df['consumption_pred'])
rmse = np.sqrt(mean_squared_error(score_df['consumption_true'], score_df['consumption_pred']))
mape = mean_absolute_percentage_error(score_df['consumption_true'], score_df['consumption_pred'])

print("\n--- 12-Month Forecast Test Set Performance ---")
print(f"Final Test MAE:  {mae:.2f}")
print(f"Final Test RMSE: {rmse:.2f}")
print(f"Final Test MAPE: {mape:.2f}")

Loading true values from monthly_df_test.csv...

Successfully merged 1344 rows for scoring.

--- 12-Month Forecast Test Set Performance ---
Final Test MAE:  822.12
Final Test RMSE: 1006.91
Final Test MAPE: 2.21
